In [55]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import re
import math
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
import lightgbm as lgb
import xgboost as xgb
import catboost as cab

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, StackingRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, StackingClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
from sklearn.preprocessing import KBinsDiscretizer
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import warnings
import json 
import pickle
warnings.filterwarnings('ignore')

数据处理

In [56]:
df_train = pd.read_csv('data/sessions_train.csv')
df_train

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE
...,...,...,...
3606244,['B086CYFSKW' 'B0874F9859' 'B086CYFSKW'],B07B5TYD76,IT
3606245,['B09NRZKZ7V' 'B08WJTPV93'],B08L1P4C3D,IT
3606246,['B085JFX7MP' 'B085JGHW8R'],B01MPWVD44,IT
3606247,['B00B0UING2' 'B00B0UING2'],B00D3HYEZ4,IT


In [57]:
df_test_task1_phase1 = pd.read_csv('data/sessions_test_task1.csv')
df_test_task1_phase1

,prev_items,locale
0,['B08V12CT4C' 'B08V1KXBQD' 'B01BVG1XJS' 'B09VC...,DE
1,['B00R9R5ND6' 'B00R9RZ9ZS' 'B00R9RZ9ZS'],DE
2,['B07YSRXJD3' 'B07G7Q5N6G' 'B08C9Q7QVK' 'B07G7...,DE
3,['B08KQBYV43' '3955350843' '3955350843' '39553...,DE
4,['B09FPTCWMC' 'B09FPTQP68' 'B08HMRY8NG' 'B08TB...,DE
...,...,...
316966,['B077SZ2C3Y' 'B0B14M3VZX'],UK
316967,['B08KFHDPY9' 'B0851KTSRZ' 'B08KFHDPY9' 'B0851...,UK
316968,['B07PY1N81F' 'B07Q1Z8SQN' 'B07PY1N81F' 'B07Q1...,UK
316969,['B01MCQMORK' 'B09JYZ325W'],UK


In [58]:
df_test_task1_phase2 = pd.read_csv('data/phase2/sessions_test_task1.csv')
df_test_task1_phase2

,prev_items,locale
0,['B087VLP2RT' 'B09BRQSHYH' 'B099KW4ZLV'],DE
1,['B08XW4W667' 'B096VMCJYF' 'B096VMCJYF'],DE
2,['B09Z4T2GJ3' 'B09Z3FBXMB' 'B0936K9LTJ' 'B09Z4...,DE
3,['B07T6Y2HG7' 'B07T2NBLX9' 'B07Y1G5F3Y'],DE
4,['B0B2DRKZ6X' 'B0B2DRKZ6X' 'B0B2DRKZ6X'],DE
...,...,...
316967,['B078RJX3CC' 'B07GKM97YF'],UK
316968,['B01LX5Y7RG' 'B00M35Y2J0' 'B0BFR9D1Y2' 'B09BB...,UK
316969,['B09HGRXXTM' 'B08VDNCZT9'],UK
316970,['B089CVQ2FS' 'B089CVQ2FS'],UK


In [59]:
df_test_task2_phase1 = pd.read_csv('data/sessions_test_task2.csv')
df_test_task2_phase1

,prev_items,locale
0,['B08GYKNCCP' 'B08HCPTMJG' 'B08HCHS64Y'],ES
1,['B08NYF9MBQ' 'B085NGXGWM'],ES
2,['B091FL1QFK' 'B0B1DG29F4'],ES
3,['B004APAHCW' 'B07JMF49HN' 'B004APAHCW' 'B07JM...,ES
4,['B09YM11D4T' 'B0B12QWP5G' 'B08YMT6Q4X' 'B09V1...,ES
...,...,...
34683,['B095W5YY7B' 'B09SGL4XKB'],IT
34684,['B0BDJZWPLQ' 'B0BDJZWPLQ'],IT
34685,['B07HKMB622' 'B00FBESXOK' 'B07HKMB622' 'B00FB...,IT
34686,['B09V4J39QY' 'B0BJMSH4VX' 'B0BJMRVQFT' 'B0BJM...,IT


In [60]:
df_test_task2_phase2 = pd.read_csv('data/phase2/sessions_test_task2.csv')
df_test_task2_phase2

,prev_items,locale
0,['B07GTS7SWK' 'B07GTS7SWK'],ES
1,['B0B33YWVHR' '849988993X' 'B09QQG85HM' 'B0BJZ...,ES
2,['B08FMPXDTJ' 'B0B4612MTM' 'B0B45YR21M' 'B0B4F...,ES
3,['B07R3W4XQ7' 'B07R3FB5B5'],ES
4,['B09V4KBWPL' 'B09G9FTLPB'],ES
...,...,...
34685,['B089F3YK6X' 'B01BBW9TEI' 'B01BBW9Q46'],IT
34686,['B078XYC42V' 'B09TW31SND'],IT
34687,['B07ZG3V3X2' 'B09PGKF9Y8'],IT
34688,['B0B3JRGZLM' 'B09SHQJ8NV'],IT


In [61]:
df_test_task3_phase1 = pd.read_csv('data/sessions_test_task3.csv')
df_test_task3_phase1

,prev_items,locale
0,['B082DLM3NZ' 'B089X86H73'],ES
1,['B071WPLND2' 'B08TMJ9SDZ' 'B07XRCLVYG'],ES
2,['B094V8G54H' 'B094V97YV8'],ES
3,['B0B3DQXY57' 'B0B6W3GGTM'],ES
4,['B0765BPD7T' 'B00V4PQY3C' 'B09HWV4MBK'],ES
...,...,...
56416,['B08GNG5FMW' 'B08Q7MJW8W'],UK
56417,['B09YH16XH1' 'B09YGY96ZM'],UK
56418,['B00EXKSNNE' 'B005DBORH8' 'B005DBORCS' 'B005D...,UK
56419,['B007CJVZ1A' 'B07GCSPHNK' 'B07GCVF3N3'],UK


In [62]:
df_test_task3_phase2 = pd.read_csv('data/phase2/sessions_test_task3.csv')
df_test_task3_phase2

,prev_items,locale
0,['B0BF9JMVDG' 'B01ET9V90M'],ES
1,['B09QQG85HM' 'B09J4T4JF5'],ES
2,['B09NSKDG4K' 'B09YY6J1ZM'],ES
3,['B09B7NYDJ7' 'B09B7NYDJ7'],ES
4,['B0B6J17LK4' 'B0B6R7X6GY' 'B07HXY5SGH'],ES
...,...,...
56417,['B08S8N3YG4' 'B00HRRANWY'],UK
56418,['B09F5L55QV' 'B0B8BRLGY4' 'B00IKGH2TI' 'B00IK...,UK
56419,['B09FY2WLZ3' 'B09FY4F65Y'],UK
56420,['B001G4L9KO' 'B0BG373773' 'B001G4L9KO' 'B07Z2...,UK


In [63]:
def str2list(x):
    x = x.replace('[', '').replace(']', '').replace("'", '').replace('\n', ' ').replace('\r', ' ')
    l = [i for i in x.split() if i]
    return l

In [64]:
df_train['prev_items'] = df_train['prev_items'].apply(lambda x: str2list(x))
df_test_task1_phase1['prev_items'] = df_test_task1_phase1['prev_items'].apply(lambda x: str2list(x))
df_test_task1_phase2['prev_items'] = df_test_task1_phase2['prev_items'].apply(lambda x: str2list(x))
df_test_task2_phase1['prev_items'] = df_test_task2_phase1['prev_items'].apply(lambda x: str2list(x))
df_test_task2_phase2['prev_items'] = df_test_task2_phase2['prev_items'].apply(lambda x: str2list(x))
df_test_task3_phase1['prev_items'] = df_test_task3_phase1['prev_items'].apply(lambda x: str2list(x))
df_test_task3_phase2['prev_items'] = df_test_task3_phase2['prev_items'].apply(lambda x: str2list(x))

In [65]:
df_train['last_item'] = df_train['prev_items'].apply(lambda x: x[-1])
df_test_task1_phase1['last_item'] = df_test_task1_phase1['prev_items'].apply(lambda x: x[-1])
df_test_task1_phase2['last_item'] = df_test_task1_phase2['prev_items'].apply(lambda x: x[-1])
df_test_task2_phase1['last_item'] = df_test_task2_phase1['prev_items'].apply(lambda x: x[-1])
df_test_task2_phase2['last_item'] = df_test_task2_phase2['prev_items'].apply(lambda x: x[-1])
df_test_task3_phase1['last_item'] = df_test_task3_phase1['prev_items'].apply(lambda x: x[-1])
df_test_task3_phase2['last_item'] = df_test_task3_phase2['prev_items'].apply(lambda x: x[-1])

In [66]:
loc2id = {'DE': 0, 'JP': 1, 'UK': 2, 'ES': 3, 'FR': 4, 'IT': 5}

In [13]:
df_test_task1_phase1.shape, df_test_task1_phase2.shape, df_test_task2_phase1.shape, df_test_task2_phase2.shape, df_test_task3_phase1.shape, df_test_task3_phase2.shape, 

((316971, 3), (316972, 3), (34688, 3), (34690, 3), (56421, 3), (56422, 3))

In [25]:
df_test_all = pd.concat([df_test_task1_phase1, df_test_task1_phase2, df_test_task2_phase1, df_test_task2_phase2, df_test_task3_phase1, df_test_task3_phase2], axis=0)

In [26]:
df_test_all

,prev_items,locale,last_item
0,"[B08V12CT4C, B08V1KXBQD, B01BVG1XJS, B09VC5PKN...",DE,B099NQFMG7
1,"[B00R9R5ND6, B00R9RZ9ZS, B00R9RZ9ZS]",DE,B00R9RZ9ZS
2,"[B07YSRXJD3, B07G7Q5N6G, B08C9Q7QVK, B07G7Q5N6G]",DE,B07G7Q5N6G
3,"[B08KQBYV43, 3955350843, 3955350843, 395535086...",DE,3955350843
4,"[B09FPTCWMC, B09FPTQP68, B08HMRY8NG, B08TBBQ4B...",DE,B09J945WQR
...,...,...,...
56417,"[B08S8N3YG4, B00HRRANWY]",UK,B00HRRANWY
56418,"[B09F5L55QV, B0B8BRLGY4, B00IKGH2TI, B00IKGH2T...",UK,B0BB7JDWB9
56419,"[B09FY2WLZ3, B09FY4F65Y]",UK,B09FY4F65Y
56420,"[B001G4L9KO, B0BG373773, B001G4L9KO, B07Z286QPY]",UK,B07Z286QPY


In [14]:
df_test_all = pd.concat([df_test_task1_phase1, df_test_task1_phase2, df_test_task2_phase1, df_test_task2_phase2], axis=0)

In [15]:
df_train

,prev_items,next_item,locale,last_item
0,"[B09W9FND7K, B09JSPLN1M]",B09M7GY217,DE,B09JSPLN1M
1,"[B076THCGSG, B007MO8IME, B08MF65MLV, B001B4TKA0]",B001B4THSA,DE,B001B4TKA0
2,"[B0B1LGXWDS, B00AZYORS2, B0B1LGXWDS, B00AZYORS...",B0767DTG2Q,DE,B00AZYORS2
3,"[B09XMTWDVT, B0B4MZZ8MB, B0B7HZ2GWX, B09XMTWDV...",B0B4R9NN4B,DE,B0B71CHT1L
4,"[B09Y5CSL3T, B09Y5DPTXN, B09FKD61R8]",B0BGVBKWGZ,DE,B09FKD61R8
...,...,...,...,...
3606244,"[B086CYFSKW, B0874F9859, B086CYFSKW]",B07B5TYD76,IT,B086CYFSKW
3606245,"[B09NRZKZ7V, B08WJTPV93]",B08L1P4C3D,IT,B08WJTPV93
3606246,"[B085JFX7MP, B085JGHW8R]",B01MPWVD44,IT,B085JGHW8R
3606247,"[B00B0UING2, B00B0UING2]",B00D3HYEZ4,IT,B00B0UING2


召回处理带泄露

In [16]:
hot200 = df_train['next_item'].value_counts().head(200).index
hot200 = list(hot200)

In [17]:
next_item_dict = defaultdict(list)

for _, row in tqdm(df_train.iterrows(), total=len(df_train)):
    prev_items = row['prev_items']
    next_item = row['next_item']
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        next_item_dict[prev_items[0]].append(next_item)
    else:
        for i, item in enumerate(prev_items[:-1]):
            next_item_dict[item].append(prev_items[i+1])
        next_item_dict[prev_items[-1]].append(next_item)

100%|██████████| 3606249/3606249 [04:25<00:00, 13580.04it/s]


In [18]:
for _, row in tqdm(df_test_all.iterrows(), total=len(df_test_all)):
    prev_items = row['prev_items']
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        continue
    else:
        for i, item in enumerate(prev_items[:-1]):
            next_item_dict[item].append(prev_items[i+1])

100%|██████████| 703321/703321 [00:47<00:00, 14689.22it/s]


In [19]:
pickle.dump(next_item_dict, open('data/next_item_dict_awesome12_leak.pkl', 'wb'))

In [20]:
next_item_map = {}

for item in tqdm(next_item_dict):
    counter = Counter(next_item_dict[item])
    next_item_map[item] = [i[0] for i in counter.most_common(100)]

100%|██████████| 1340537/1340537 [00:16<00:00, 82167.46it/s] 


In [21]:
pickle.dump(next_item_map, open('data/next_item_map_awesome12_leak.pkl', 'wb'))

In [22]:
k = []
v = []

for item in next_item_dict:
    k.append(item)
    v.append(next_item_dict[item])
    
df_next = pd.DataFrame({'item': k, 'next_item': v})
df_next = df_next.explode('next_item').reset_index(drop=True)
df_next

,item,next_item
0,B09W9FND7K,B09JSPLN1M
1,B09W9FND7K,B09JSPLN1M
2,B09W9FND7K,B09JSPLN1M
3,B09W9FND7K,B09JSPLN1M
4,B09W9FND7K,B078WW2WN5
...,...,...
17565057,B0B77ZJKD2,B0B781WMXW
17565058,B00VZZ7VVA,B00V3MQ1PC
17565059,B00VZZ7VVA,B00V3MQ1PC
17565060,B07KKKNB32,B07KKKNB32


In [23]:
top200 = df_next['next_item'].value_counts().index.tolist()[:200]  # 所有下一跳候选集中最热门的200个

In [24]:
pickle.dump(top200, open('data/top200_awesome12_leak.pkl', 'wb'))

召回处理不带泄露

In [25]:
# 去除泄露
next_item_dict = defaultdict(list)

for _, row in tqdm(df_train.iterrows(), total=len(df_train)):
    prev_items = row['prev_items']
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        continue
    else:
        for i, item in enumerate(prev_items[:-1]):
            next_item_dict[item].append(prev_items[i+1])

100%|██████████| 3606249/3606249 [03:54<00:00, 15373.54it/s]


In [26]:
for _, row in tqdm(df_test_all.iterrows(), total=len(df_test_all)):
    prev_items = row['prev_items']
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        continue
    else:
        for i, item in enumerate(prev_items[:-1]):
            next_item_dict[item].append(prev_items[i+1])

100%|██████████| 703321/703321 [00:45<00:00, 15395.66it/s]


In [27]:
pickle.dump(next_item_dict, open('data/next_item_dict_awesome12_noleak.pkl', 'wb'))

In [28]:
next_item_map = {}

for item in tqdm(next_item_dict):
    counter = Counter(next_item_dict[item])
    next_item_map[item] = [i[0] for i in counter.most_common(100)]

100%|██████████| 1270173/1270173 [00:15<00:00, 82729.30it/s] 


In [29]:
pickle.dump(next_item_map, open('data/next_item_map_awesome12_noleak.pkl', 'wb'))

In [30]:
k = []
v = []

for item in next_item_dict:
    k.append(item)
    v.append(next_item_dict[item])
    
df_next = pd.DataFrame({'item': k, 'next_item': v})
df_next = df_next.explode('next_item').reset_index(drop=True)
df_next

,item,next_item
0,B09W9FND7K,B09JSPLN1M
1,B09W9FND7K,B09JSPLN1M
2,B09W9FND7K,B09JSPLN1M
3,B09W9FND7K,B09JSPLN1M
4,B09W9FND7K,B09W9FND7K
...,...,...
13958808,B0B77ZJKD2,B0B781WMXW
13958809,B00VZZ7VVA,B00V3MQ1PC
13958810,B00VZZ7VVA,B00V3MQ1PC
13958811,B07KKKNB32,B07KKKNB32


In [31]:
top200 = df_next['next_item'].value_counts().index.tolist()[:200]  # 所有下一跳候选集中最热门的200个

In [32]:
pickle.dump(top200, open('data/top200_awesome12_noleak.pkl', 'wb'))

本地测试

In [45]:
next_item_dict = pickle.load(open('data/next_item_dict_awesome12_noleak.pkl', 'rb'))
next_item_map = pickle.load(open('data/next_item_map_awesome12_noleak.pkl', 'rb'))
top200 = pickle.load(open('data/top200_awesome12_noleak.pkl', 'rb'))

In [46]:
df_train['next_item_prediction'] = df_train['last_item'].map(next_item_map)
df_train

,prev_items,next_item,locale,last_item,next_item_prediction
0,"[B09W9FND7K, B09JSPLN1M]",B09M7GY217,DE,B09JSPLN1M,"[B09W9FND7K, B09YS6S48R, B07KDC7PJH, B07MDZ2K4..."
1,"[B076THCGSG, B007MO8IME, B08MF65MLV, B001B4TKA0]",B001B4THSA,DE,B001B4TKA0,[B001B4THSA]
2,"[B0B1LGXWDS, B00AZYORS2, B0B1LGXWDS, B00AZYORS...",B0767DTG2Q,DE,B00AZYORS2,"[B0B1LGXWDS, B07RB1VFPB, B004N9BSQE, B08XXLDH2..."
3,"[B09XMTWDVT, B0B4MZZ8MB, B0B7HZ2GWX, B09XMTWDV...",B0B4R9NN4B,DE,B0B71CHT1L,"[B0B71GSJ2R, B0B71KXCSB, B09XMTWDVT, B09Z25LH2..."
4,"[B09Y5CSL3T, B09Y5DPTXN, B09FKD61R8]",B0BGVBKWGZ,DE,B09FKD61R8,"[B09MRVWQ3B, B09FKDPY2S]"
...,...,...,...,...,...
3606244,"[B086CYFSKW, B0874F9859, B086CYFSKW]",B07B5TYD76,IT,B086CYFSKW,"[B0874F9859, B08FJQFPLM]"
3606245,"[B09NRZKZ7V, B08WJTPV93]",B08L1P4C3D,IT,B08WJTPV93,"[B08WJTPV93, 8847224209, 8852225196, B0BHKZFY1..."
3606246,"[B085JFX7MP, B085JGHW8R]",B01MPWVD44,IT,B085JGHW8R,"[B085JGHW8R, B07RYSKF2Z, B01MPWVD44, B09RZJZ7Y..."
3606247,"[B00B0UING2, B00B0UING2]",B00D3HYEZ4,IT,B00B0UING2,"[B00B0UING2, B00D3HYEZ4]"


In [47]:
train_preds = []
mrr = []

for _, row in tqdm(df_train.iterrows(), total=len(df_train)):
    label = row['next_item']
    pred_orig = row['next_item_prediction']
    pred = pred_orig
    prev_items = row['prev_items']
    if type(pred) == float:  # pred=nan, 训练集的last_item在之间没有统计出来下一跳，此时用最热门代替
        pred = top200[:100]
    else:
        if len(pred_orig) < 100:  # 小于100，从最热门选
            for i in top200:
                if i not in pred_orig and i not in prev_items:
                    pred.append(i)
                if len(pred) >= 100:
                    break
        else:
            pred = pred[:100]
    try:
        pred_result = pred.index(label)
        mrr.append(1 / (pred_result + 1))
    except:
        mrr.append(0)
    train_preds.append(pred)

np.mean(mrr)

100%|██████████| 3606249/3606249 [06:35<00:00, 9115.07it/s] 


0.2130036983866915

In [48]:
pickle.dump(train_preds, open('./data/train_preds_next_one_awesome12_noleak.pkl', 'wb'))

远程测试task1

In [49]:
next_item_dict = pickle.load(open('data/next_item_dict_awesome12_leak.pkl', 'rb'))
next_item_map = pickle.load(open('data/next_item_map_awesome12_leak.pkl', 'rb'))
top200 = pickle.load(open('data/top200_awesome12_leak.pkl', 'rb'))

In [50]:
df_test_task1_phase2['next_item_prediction'] = df_test_task1_phase2['last_item'].map(next_item_map)
df_test_task1_phase2

,prev_items,locale,last_item,next_item_prediction
0,"[B087VLP2RT, B09BRQSHYH, B099KW4ZLV]",DE,B099KW4ZLV,"[B07SDFLVKD, B091CK241X, B0B9GJLV2D, B0BGC82WV..."
1,"[B08XW4W667, B096VMCJYF, B096VMCJYF]",DE,B096VMCJYF,"[B084CB7GX9, B08XW4W667, B09YD8XV6M, B004P4OF1..."
2,"[B09Z4T2GJ3, B09Z3FBXMB, B0936K9LTJ, B09Z4PYG8Q]",DE,B09Z4PYG8Q,"[B09Z4T2GJ3, B09Z3FBXMB, B09Z4PZQBF, B0936K9LT..."
3,"[B07T6Y2HG7, B07T2NBLX9, B07Y1G5F3Y]",DE,B07Y1G5F3Y,"[B07T2NBLX9, B07Y1KLF25, B07T6Y2HG7, B07HQ83TF..."
4,"[B0B2DRKZ6X, B0B2DRKZ6X, B0B2DRKZ6X]",DE,B0B2DRKZ6X,"[B0B2DRKZ6X, B0B2JY9THB, B08YK8FQJ8, B08SHZHRQ..."
...,...,...,...,...
316967,"[B078RJX3CC, B07GKM97YF]",UK,B07GKM97YF,"[B07GKP2LCF, B07GKYSHB4, B016RAAUEM, B006DDGCI..."
316968,"[B01LX5Y7RG, B00M35Y2J0, B0BFR9D1Y2, B09BB3KLX...",UK,B000FHC0QK,"[B00M35Y326, B000FHC0QK, B08B395NHL, B091DWY6C..."
316969,"[B09HGRXXTM, B08VDNCZT9]",UK,B08VDNCZT9,"[B08VDGMBGP, B08VDSL596, B08VD5DC5L, B08VDNCZT..."
316970,"[B089CVQ2FS, B089CVQ2FS]",UK,B089CVQ2FS,"[B089CZWB4C, B08T1ZJYHV, B08DTYFYGP, B08W2JJZB..."


In [51]:
test_preds = []

for _, row in tqdm(df_test_task1_phase2.iterrows(), total=len(df_test_task1_phase2)):
    pred_orig = row['next_item_prediction']
    pred = pred_orig
    prev_items = row['prev_items']
    if type(pred) == float:  # pred=nan, 测试集的last_item在之间没有统计出来下一跳，此时用最热门代替
        pred = top200[:100]
    else:
        if len(pred_orig) < 100:  # 小于100，从最热门选
            for i in top200:
                if i not in pred_orig and i not in prev_items:
                    pred.append(i)
                if len(pred) >= 100:
                    break
        else:
            pred = pred[:100]
    test_preds.append(pred)

100%|██████████| 316972/316972 [00:38<00:00, 8319.01it/s] 


In [52]:
pickle.dump(test_preds, open('./data/test_preds_next_one_task1_phase2_awesome12_leak.pkl', 'wb'))

In [53]:
df_test_task1_phase2['next_item_prediction'] = test_preds
df_test_task1_phase2['next_item_prediction'].apply(len).describe()

count    316972.0
mean        100.0
std           0.0
min         100.0
25%         100.0
50%         100.0
75%         100.0
max         100.0
Name: next_item_prediction, dtype: float64

In [54]:
df_test_task1_phase2[['locale', 'next_item_prediction']].to_parquet('output/phase2_submission_task1_rule_next_one_awesome12_leak.parquet', engine='pyarrow')

远程测试task2

In [33]:
next_item_dict = pickle.load(open('data/next_item_dict_awesome12_leak.pkl', 'rb'))
next_item_map = pickle.load(open('data/next_item_map_awesome12_leak.pkl', 'rb'))
top200 = pickle.load(open('data/top200_awesome12_leak.pkl', 'rb'))

In [34]:
df_test_task2_phase2['next_item_prediction'] = df_test_task2_phase2['last_item'].map(next_item_map)
df_test_task2_phase2

,prev_items,locale,last_item,next_item_prediction
0,"[B07GTS7SWK, B07GTS7SWK]",ES,B07GTS7SWK,"[B07GTS7SWK, B0745795KF, B074581R8Y, B074574XP..."
1,"[B0B33YWVHR, 849988993X, B09QQG85HM, B0BJZLJXR...",ES,B09K7TDY1H,"[B09M8LNB61, B09K7TDY1H, B09J4T4JF5, B08BRGH7S..."
2,"[B08FMPXDTJ, B0B4612MTM, B0B45YR21M, B0B4F2L38...",ES,B0B45YR21M,"[B0B4612MTM, B0B461V1RB, B0B4614XL6, B0B461KYH..."
3,"[B07R3W4XQ7, B07R3FB5B5]",ES,B07R3FB5B5,"[B07R4VG9X3, B07R4WKSSV, B07R3W4XQ7, B07P5ZKXF..."
4,"[B09V4KBWPL, B09G9FTLPB]",ES,B09G9FTLPB,"[B09G9LF91K, B09G97SLGS, B09G9SC2VR, B09V4KBWP..."
...,...,...,...,...
34685,"[B089F3YK6X, B01BBW9TEI, B01BBW9Q46]",IT,B01BBW9Q46,"[B01BBW9TEI, B09QH4W4LS, B01F545TZQ, B01BBW9OA..."
34686,"[B078XYC42V, B09TW31SND]",IT,B09TW31SND,"[B08DYGN5W7, B00ECBKODQ, B078XYC42V, B01KLXFBV..."
34687,"[B07ZG3V3X2, B09PGKF9Y8]",IT,B09PGKF9Y8,"[B07ZH7HYDX, B09PGKF9Y8, B07KFBBYLS, B00BNUCFK..."
34688,"[B0B3JRGZLM, B09SHQJ8NV]",IT,B09SHQJ8NV,"[B0B2PFFCBH, B09W2Z395Q, B0B7HWCVNT, B09XMR51P..."


In [35]:
test_preds = []

for _, row in tqdm(df_test_task2_phase2.iterrows(), total=len(df_test_task2_phase2)):
    pred_orig = row['next_item_prediction']
    pred = pred_orig
    prev_items = row['prev_items']
    if type(pred) == float:  # pred=nan, 测试集的last_item在之间没有统计出来下一跳，此时用最热门代替
        pred = top200[:100]
    else:
        if len(pred_orig) < 100:  # 小于100，从最热门选
            for i in top200:
                if i not in pred_orig and i not in prev_items:
                    pred.append(i)
                if len(pred) >= 100:
                    break
        else:
            pred = pred[:100]
    test_preds.append(pred)

100%|██████████| 34690/34690 [00:04<00:00, 8623.32it/s] 


In [36]:
pickle.dump(test_preds, open('./data/test_preds_next_one_task2_phase2_awesome12_leak.pkl', 'wb'))

In [37]:
df_test_task2_phase2['next_item_prediction'] = test_preds
df_test_task2_phase2['next_item_prediction'].apply(len).describe()

count    34690.0
mean       100.0
std          0.0
min        100.0
25%        100.0
50%        100.0
75%        100.0
max        100.0
Name: next_item_prediction, dtype: float64

In [38]:
df_test_task2_phase2[['locale', 'next_item_prediction']].to_parquet('output/phase2_submission_task2_rule_next_one_awesome12_leak.parquet', engine='pyarrow')

11